__https://docs.python.org/3/library/sqlite3.html#sqlite3-tutorial__

In [1]:
import sqlite3
con = sqlite3.connect("tutorial.db")

In [2]:
# In order to execute SQL statements and fetch results from SQL queries, 
# we will need to use a database cursor.
cur = con.cursor()

In [3]:
cur.execute("CREATE TABLE movie(title, year, score)")

In [4]:
# We can verify that the new table has been created by querying the sqlite_master table 
# built-in to SQLite, which should now contain an entry for the movie table definition 
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()

('movie',)

In [5]:
# We can see that the table has been created, as the query returns a tuple 
# containing the table’s name. If we query sqlite_master for a non-existent 
# table spam, res.fetchone() will return None:
res = cur.execute("select name from sqlite_master where name = 'spam'")
res.fetchone() is None

True

In [8]:
cur.execute("""insert into movie values
            ('Monty python and the Holy Grail',1975,8.2),
            ('And now for something completely different',1971,7.5)
""")


In [9]:
# The INSERT statement implicitly opens a transaction, 
# which needs to be committed before changes are saved in the database
con.commit()

In [10]:
# We can verify that the data was inserted correctly by executing a SELECT query.
# call res.fetchall() to return all resulting rows
res = cur.execute("select score from movie")
res.fetchall()

# The result is a list of two tuples, one per row, each containing that row’s score value.

[(8.2,), (7.5,)]

In [11]:
# Now, insert three more rows by calling cur.executemany(...):

data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
con.commit()  # Remember to commit the transaction after executing INSERT.

# Notice that ? placeholders are used to bind data to the query. 
# Always use placeholders instead of string formatting to bind Python values 
# to SQL statements, to avoid SQL injection attacks

In [12]:
# We can verify that the new rows were inserted by executing a SELECT query, 
# this time iterating over the results of the query:
for row in cur.execute("select year, title from movie order by year"):
    print(row)


(1971, 'And now for something completely different')
(1975, 'Monty python and the Holy Grail')
(1979, "Monty Python's Life of Brian")
(1982, 'Monty Python Live at the Hollywood Bowl')
(1983, "Monty Python's The Meaning of Life")


In [18]:
# Finally, verify that the database has been written to disk by calling con.close() 
# to close the existing connection, opening a new one, creating a new cursor, 
# then querying the database:
con.close()
new_con = sqlite3.Connection("tutorial.db")
new_cur = new_con.cursor()
res = new_cur.execute("select title, year from movie order by score desc")
title, year = res.fetchone()
print(f'The highest scoring monty python movie is {title!r}, released in {year}')


The highest scoring monty python movie is 'Monty python and the Holy Grail', released in 1975
